In [70]:
import numpy as np
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

import ast

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
import warnings
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split 

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [71]:
charging_set = pd.read_csv("data/charging_sessions.csv")

In [72]:
# get overview of data
print("columns:")
print(charging_set.columns)
print("-------------------------------------------------------------------------")
print("info():")
print(charging_set.info())
print("-------------------------------------------------------------------------")
print("describe():")
print(charging_set.describe())

columns:
Index(['Unnamed: 0', 'id', 'connectionTime', 'disconnectTime',
       'doneChargingTime', 'kWhDelivered', 'sessionID', 'siteID', 'spaceID',
       'stationID', 'timezone', 'userID', 'userInputs'],
      dtype='object')
-------------------------------------------------------------------------
info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        66450 non-null  int64  
 1   id                66450 non-null  object 
 2   connectionTime    66450 non-null  object 
 3   disconnectTime    66450 non-null  object 
 4   doneChargingTime  62362 non-null  object 
 5   kWhDelivered      66450 non-null  float64
 6   sessionID         66450 non-null  object 
 7   siteID            66450 non-null  int64  
 8   spaceID           66450 non-null  object 
 9   stationID         66450 non-null  object 
 10  timezone    

In [73]:
charging_set.head(10)

,Unnamed: 0,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
3,3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
4,4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil..."
5,5,5e23b149f9af8b5fe4b973d4,2020-01-02 14:00:13+00:00,2020-01-02 23:00:41+00:00,2020-01-02 20:13:21+00:00,13.370,1_1_194_821_2020-01-02 14:00:05.241573,1,AG-1F13,1-1-194-821,America/Los_Angeles,3519.0,"[{'WhPerMile': 600, 'kWhRequested': 24.0, 'mil..."
6,6,5e23b149f9af8b5fe4b973d5,2020-01-02 14:09:14+00:00,2020-01-03 01:57:58+00:00,2020-01-03 00:59:21+00:00,43.477,1_1_178_817_2020-01-02 14:09:14.451827,1,AG-1F09,1-1-178-817,America/Los_Angeles,933.0,"[{'WhPerMile': 385, 'kWhRequested': 65.45, 'mi..."
7,7,5e23b149f9af8b5fe4b973d6,2020-01-02 14:17:32+00:00,2020-01-03 00:44:54+00:00,2020-01-02 20:22:40+00:00,32.441,1_1_191_804_2020-01-02 14:17:32.329535,1,AG-4F34,1-1-191-804,America/Los_Angeles,3737.0,"[{'WhPerMile': 300, 'kWhRequested': 45.0, 'mil..."
8,8,5e23b149f9af8b5fe4b973d7,2020-01-02 14:25:38+00:00,2020-01-02 19:36:05+00:00,2020-01-02 18:38:39+00:00,13.264,1_1_194_826_2020-01-02 14:25:37.578692,1,AG-1F11,1-1-194-826,America/Los_Angeles,419.0,"[{'WhPerMile': 400, 'kWhRequested': 40.0, 'mil..."
9,9,5e23b149f9af8b5fe4b973d8,2020-01-02 14:27:40+00:00,2020-01-02 20:01:48+00:00,2020-01-02 19:17:46+00:00,13.285,1_1_178_823_2020-01-02 14:27:39.582337,1,AG-1F08,1-1-178-823,America/Los_Angeles,651.0,"[{'WhPerMile': 400, 'kWhRequested': 20.0, 'mil..."


# TODO

- check validity and meaning of data (userInput)
- remove irrelevant columns
- split userInput in own columns
- check for duplicates
- check for None-values (userID)
- split userID into binary dummy (isRegistered)
- check for None-values for isRegistered = 1 (remove None-values for registered users)
- check data types of columns
- look at columns with multiple data types (remove wrong entries)
- check ranges of columns & unique values

# Data Cleanup

## Step 1: check validity of columns

- remove unnecessary "unnamed" column
- sessionID does not bring any relevant information for our analysis

In [74]:
# remove irrelevant columns
charging_set.drop('Unnamed: 0', axis=1, inplace=True)
charging_set.drop('sessionID', axis=1, inplace=True) # does not bring relevant information for our analysis
charging_set.head(3)

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,timezone,userID,userInputs
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."


In [75]:
# split userInputs into own columns
def parse_user_inputs(row):
    try:
        # Parse the row as a Python object (e.g., list of dicts)
        parsed = ast.literal_eval(row) if pd.notnull(row) else []
        return parsed[0] if isinstance(parsed, list) and len(parsed) > 0 else {}
    except (ValueError, SyntaxError):
        return {}

# Parse 'userInputs' column in charging_set
charging_set['userInputs'] = charging_set['userInputs'].apply(parse_user_inputs)

# Specify the desired features to extract
desired_features = [
    'WhPerMile', 'kWhRequested', 'milesRequested',
    'minutesAvailable', 'modifiedAt', 'paymentRequired',
    'requestedDeparture'
]

# Extract desired features and create new columns
for feature in desired_features:
    charging_set[feature] = charging_set['userInputs'].apply(
        lambda x: x.get(feature) if isinstance(x, dict) else None
    )
# TODO: has to be commented so a cell further down in this notebook can run (it needs userInputs)
charging_set = charging_set.drop(columns=['userInputs'])
charging_set

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,timezone,userID,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,250.0,25.00,100.0,463.0,"Thu, 02 Jan 2020 13:09:39 GMT",True,"Thu, 02 Jan 2020 20:51:54 GMT"
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,280.0,70.00,250.0,595.0,"Thu, 02 Jan 2020 13:37:11 GMT",True,"Thu, 02 Jan 2020 23:31:50 GMT"
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,400.0,8.00,20.0,60.0,"Thu, 02 Jan 2020 13:57:17 GMT",True,"Thu, 02 Jan 2020 14:56:35 GMT"
3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,400.0,8.00,20.0,65.0,"Thu, 02 Jan 2020 14:00:03 GMT",True,"Thu, 02 Jan 2020 15:04:58 GMT"
4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,400.0,16.00,40.0,504.0,"Thu, 02 Jan 2020 14:00:13 GMT",True,"Thu, 02 Jan 2020 22:24:01 GMT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66445,5d574ad2f9af8b4c10c03652,2019-07-31 18:08:04+00:00,2019-07-31 23:29:18+00:00,2019-07-31 23:30:18+00:00,28.787,1,AG-3F27,1-1-179-809,America/Los_Angeles,393.0,240.0,31.20,130.0,355.0,"Wed, 31 Jul 2019 18:08:23 GMT",True,"Thu, 01 Aug 2019 00:03:04 GMT"
66446,5d574ad2f9af8b4c10c03653,2019-07-31 18:40:41+00:00,2019-08-01 00:59:42+00:00,2019-07-31 21:44:23+00:00,7.787,1,AG-3F30,1-1-179-810,America/Los_Angeles,220.0,333.0,6.66,20.0,455.0,"Wed, 31 Jul 2019 18:41:02 GMT",True,"Thu, 01 Aug 2019 02:15:41 GMT"
66447,5d574ad2f9af8b4c10c03654,2019-07-31 19:04:40+00:00,2019-07-31 22:44:22+00:00,2019-07-31 22:45:21+00:00,11.274,1,AG-4F51,1-1-191-795,America/Los_Angeles,1974.0,333.0,19.98,60.0,184.0,"Wed, 31 Jul 2019 19:04:57 GMT",True,"Wed, 31 Jul 2019 22:08:40 GMT"
66448,5d574ad2f9af8b4c10c03655,2019-07-31 19:19:47+00:00,2019-08-01 00:34:51+00:00,2019-07-31 21:25:30+00:00,11.589,1,AG-4F43,1-1-191-778,America/Los_Angeles,942.0,275.0,22.00,80.0,60.0,"Wed, 31 Jul 2019 19:20:10 GMT",True,"Wed, 31 Jul 2019 20:19:47 GMT"


In [76]:
# check for duplicates
duplicate_counter = charging_set.duplicated().sum()
duplicates = charging_set[charging_set.duplicated()]
print("sum of duplicates:", duplicate_counter)
charging_set.drop_duplicates()

sum of duplicates: 1413


,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,timezone,userID,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,250.0,25.00,100.0,463.0,"Thu, 02 Jan 2020 13:09:39 GMT",True,"Thu, 02 Jan 2020 20:51:54 GMT"
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,280.0,70.00,250.0,595.0,"Thu, 02 Jan 2020 13:37:11 GMT",True,"Thu, 02 Jan 2020 23:31:50 GMT"
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,400.0,8.00,20.0,60.0,"Thu, 02 Jan 2020 13:57:17 GMT",True,"Thu, 02 Jan 2020 14:56:35 GMT"
3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,400.0,8.00,20.0,65.0,"Thu, 02 Jan 2020 14:00:03 GMT",True,"Thu, 02 Jan 2020 15:04:58 GMT"
4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,400.0,16.00,40.0,504.0,"Thu, 02 Jan 2020 14:00:13 GMT",True,"Thu, 02 Jan 2020 22:24:01 GMT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65032,5d2fbdd3f9af8b4d0dd0d54a,2019-07-01 21:49:12+00:00,2019-07-02 01:37:18+00:00,2019-07-01 23:14:33+00:00,8.399,1,AG-3F18,1-1-179-798,America/Los_Angeles,1346.0,400.0,32.00,80.0,36.0,"Mon, 01 Jul 2019 21:49:37 GMT",True,"Mon, 01 Jul 2019 22:25:12 GMT"
65033,5d2fbdd3f9af8b4d0dd0d54b,2019-07-01 21:58:45+00:00,2019-07-02 00:39:48+00:00,2019-07-02 00:40:21+00:00,16.864,1,AG-3F20,1-1-179-794,America/Los_Angeles,364.0,400.0,40.00,100.0,234.0,"Mon, 01 Jul 2019 21:59:09 GMT",True,"Tue, 02 Jul 2019 01:52:45 GMT"
65034,5d2fbdd3f9af8b4d0dd0d54c,2019-07-01 22:02:21+00:00,2019-07-02 00:58:50+00:00,2019-07-02 00:59:23+00:00,18.335,1,AG-4F47,1-1-191-807,America/Los_Angeles,2050.0,333.0,29.97,90.0,309.0,"Mon, 01 Jul 2019 22:02:37 GMT",True,"Tue, 02 Jul 2019 03:11:21 GMT"
65035,5d2fbdd3f9af8b4d0dd0d54d,2019-07-01 22:23:44+00:00,2019-07-02 02:03:36+00:00,2019-07-02 02:04:06+00:00,22.815,1,AG-3F31,1-1-179-781,America/Los_Angeles,1626.0,200.0,34.00,170.0,99.0,"Mon, 01 Jul 2019 22:23:43 GMT",True,"Tue, 02 Jul 2019 00:02:44 GMT"


The next step is to check for NaN-values. We want to differentiate between time-data and non-time-data since we need to treat time-data differently because of the time-format. Before we look at the time-data we handle the NaN-values of the non-time-data.

In [77]:
# check for none-values
nan_counts = charging_set.isna().sum()
nan_counts_sorted = nan_counts.sort_values(ascending=False)

for column, count in nan_counts_sorted.items():
    print(f"Feature: {column}, Count of NaN-Values: {count}")

Feature: requestedDeparture, Count of NaN-Values: 17263
Feature: milesRequested, Count of NaN-Values: 17263
Feature: paymentRequired, Count of NaN-Values: 17263
Feature: modifiedAt, Count of NaN-Values: 17263
Feature: minutesAvailable, Count of NaN-Values: 17263
Feature: WhPerMile, Count of NaN-Values: 17263
Feature: kWhRequested, Count of NaN-Values: 17263
Feature: userID, Count of NaN-Values: 17263
Feature: doneChargingTime, Count of NaN-Values: 4088
Feature: id, Count of NaN-Values: 0
Feature: timezone, Count of NaN-Values: 0
Feature: spaceID, Count of NaN-Values: 0
Feature: stationID, Count of NaN-Values: 0
Feature: kWhDelivered, Count of NaN-Values: 0
Feature: disconnectTime, Count of NaN-Values: 0
Feature: connectionTime, Count of NaN-Values: 0
Feature: siteID, Count of NaN-Values: 0


We observe a large number of NaN-values in the columns that were separated from the column "userInputs". It is very noticeable that the number of NaN-values of the "userInputs"-columns is the same as the number of the NaN-values in the "userID"-column. Accordingly, we assume that the NaN-values in the "userInputs"-columns originate from users not being registered. In the following, we check if our assumption is correct:

In [78]:
# check if users not being registered is the reason for NaN-values in userInputs-columns
user_inputs = ["requestedDeparture", "milesRequested", "paymentRequired", "modifiedAt", "minutesAvailable", "WhPerMile", "kWhRequested"]
def check_user_inputs_nan_condition(user_inputs_columns):
    """
    check_userInputs_nan_condition checks if userID is NaN when unserInputs-columns are NaN
    
    :param user_inputs_columns: all columns that were stored in the former "userInputs"-column
    :return: 
    """
    # iterate over each row and compare userID-value to userInputs-values
    for idx, row in charging_set.iterrows():
        user_id_value = row["userID"]

        for user_input_column in user_inputs_columns:
            user_input_value = row[user_input_column]

            # if userID is NaN but userInput_column is not NaN, return false
            if pd.isna(user_input_value) and not pd.isna(user_id_value):
                return False

    # if no violation of condition is found, return true
    return True

print("unregistered users are the reason for NaN-values in userInputs-columns:", check_user_inputs_nan_condition(user_inputs))

# check if we can use 0 as a placeholder for NaN values for "userID" and "userInputs"-columns
columns_contain_nan = ["userID", "milesRequested", "paymentRequired", "minutesAvailable", "WhPerMile", "kWhRequested"] # INFO: we excluded time data here, since we need to treat it differently beacause of the time-format
contains_zero_value = (charging_set[columns_contain_nan] == 0).any()
print("userID contains a zero value:\n", contains_zero_value)

columns_contain_nan_wo_zero_values = [col for col in columns_contain_nan if not (charging_set[col] == 0).any()]
columns_contain_nan_w_zero_values = [col for col in columns_contain_nan if (charging_set[col] == 0).any()]

# fill NaN-values with 0 in columns that do not contain zero values, for the rest of the columns, use -1
charging_set[columns_contain_nan_wo_zero_values] = charging_set[columns_contain_nan_wo_zero_values].fillna(0)
charging_set[columns_contain_nan_w_zero_values] = charging_set[columns_contain_nan_w_zero_values].fillna(-1)

unregistered users are the reason for NaN-values in userInputs-columns: True
userID contains a zero value:
 userID              False
milesRequested       True
paymentRequired     False
minutesAvailable    False
WhPerMile           False
kWhRequested         True
dtype: bool


Now we have a look at the time data. Since all the time-data is still in a human-unreadable-format, first we convert the data into a for us readable format. After that we take care of the NaN-values. 

In [79]:
# Convert the columns to datetime format
charging_set['connectionTime'] = pd.to_datetime(charging_set['connectionTime'])
charging_set['disconnectTime'] = pd.to_datetime(charging_set['disconnectTime'])
charging_set['doneChargingTime'] = pd.to_datetime(charging_set['doneChargingTime'])
charging_set['modifiedAt'] = pd.to_datetime(charging_set['modifiedAt'])
charging_set['requestedDeparture'] = pd.to_datetime(charging_set['requestedDeparture'])

### split connectionTime in separate day and time columns
charging_set['connectionTime_date'] = charging_set['connectionTime'].dt.date
charging_set['connectionTime_time'] = charging_set['connectionTime'].dt.time

### split disconnectTime in separate day and time columns
charging_set['disconnectTime_date'] = charging_set['disconnectTime'].dt.date
charging_set['disconnectTime_time'] = charging_set['disconnectTime'].dt.time

### split doneChargingTime in separate day and time columns
charging_set['doneChargingTime_date'] = charging_set['doneChargingTime'].dt.date
charging_set['doneChargingTime_time'] = charging_set['doneChargingTime'].dt.time


### split disconnectTime in separate day and time columns
charging_set['modifiedAt_date'] = charging_set['modifiedAt'].dt.date
charging_set['modifiedAt_time'] = charging_set['modifiedAt'].dt.time

### split disconnectTime in separate day and time columns
charging_set['requestedDeparture_date'] = charging_set['requestedDeparture'].dt.date
charging_set['requestedDeparture_time'] = charging_set['requestedDeparture'].dt.time

In [80]:
### convert date columns into separate year, month and day columns

# Das muss rein, weil sonst kommt ein Fehler
charging_set['connectionTime_date'] = pd.to_datetime(charging_set['connectionTime_date'])
charging_set['disconnectTime_date'] = pd.to_datetime(charging_set['disconnectTime_date'])
charging_set['doneChargingTime_date'] = pd.to_datetime(charging_set['doneChargingTime_date'])
charging_set['modifiedAt_date'] = pd.to_datetime(charging_set['modifiedAt_date'])
charging_set['requestedDeparture_date'] = pd.to_datetime(charging_set['requestedDeparture_date'])

charging_set['connectionTime_year'] = charging_set['connectionTime_date'].dt.year
charging_set['connectionTime_month'] = charging_set['connectionTime_date'].dt.month
charging_set['connectionTime_day'] = charging_set['connectionTime_date'].dt.day

charging_set['disconnectTime_year'] = charging_set['disconnectTime_date'].dt.year
charging_set['disconnectTime_month'] = charging_set['disconnectTime_date'].dt.month
charging_set['disconnectTime_day'] = charging_set['disconnectTime_date'].dt.day

charging_set['doneChargingTime_year'] = charging_set['doneChargingTime_date'].dt.year
charging_set['doneChargingTime_month'] = charging_set['doneChargingTime_date'].dt.month
charging_set['doneChargingTime_day'] = charging_set['doneChargingTime_date'].dt.day

charging_set['modifiedAt_year'] = charging_set['modifiedAt_date'].dt.year
charging_set['modifiedAt_month'] = charging_set['modifiedAt_date'].dt.month
charging_set['modifiedAt_day'] = charging_set['modifiedAt_date'].dt.day
# charging_set['modifiedAt_weekday'] = charging_set['modifiedAt_date'].dt.weekday

charging_set['requestedDeparture_year'] = charging_set['requestedDeparture_date'].dt.year
charging_set['requestedDeparture_month'] = charging_set['requestedDeparture_date'].dt.month
charging_set['requestedDeparture_day'] = charging_set['requestedDeparture_date'].dt.day
# charging_set['requestedDeparture_weekday'] = charging_set['requestedDeparture_date'].dt.weekday

In [81]:
def calculate_hours():
    # Berechne die Ladezeit als Differenz zwischen disconnectTime und connectionTime
    charging_set["chargingDurationTemp"] = charging_set["disconnectTime"] - charging_set["connectionTime"]
    
    # Konvertiere die Differenz in Stunden
    charging_set['chargingDuration'] = charging_set["chargingDurationTemp"].apply(
        lambda x: x.total_seconds() / 3600
    )
    
    # Entferne die temporäre Spalte
    charging_set.drop(columns=["chargingDurationTemp"], inplace=True)
    
    return charging_set

# Aufruf der Funktion
charging_set = calculate_hours()

In [82]:
# clean "connectionTime", "disconnectTime", "doneChargingTime" from dataframe
charging_set = charging_set.drop(columns=["connectionTime", "disconnectTime", "doneChargingTime"])

In [84]:
### check data types of each column
# Find the unique types
# TODO: add userInputs columns into here
id_types = charging_set["id"].apply(type).unique()
kWhDelivered_types = charging_set["kWhDelivered"].apply(type).unique()
siteID_types = charging_set["siteID"].apply(type).unique()
spaceID_types = charging_set["spaceID"].apply(type).unique()
stationID_types = charging_set["stationID"].apply(type).unique()
timezone_types = charging_set["timezone"].apply(type).unique()
userID_types = charging_set["userID"].apply(type).unique()

# Print the unique types
print("id types:", id_types)
print("kWhDelivered types:", kWhDelivered_types)
print("siteID types:", siteID_types)
print("spaceID types:", spaceID_types)
print("stationID types:", stationID_types)
print("timezone types:", timezone_types)
print("userID types:", userID_types)

id types: [<class 'str'>]
kWhDelivered types: [<class 'float'>]
siteID types: [<class 'int'>]
spaceID types: [<class 'str'>]
stationID types: [<class 'str'>]
timezone types: [<class 'str'>]
userID types: [<class 'float'>]


In [87]:
##### check ranges of relevant column values
# TODO: fix dataset such that there is only one datatype for one column, then this cell runs smoothly
# time data is still in human unreadable format
# print("connectionTime data range:\n", np.sort(charging_set["connectionTime"].unique()))
# print("disconnectTime data range:\n", np.sort(charging_set["disconnectTime"].unique()))
# print("doneChargingTime data range:\n", np.sort(charging_set["doneChargingTime"].unique()))
print("kWhDelivered data range:\n", np.sort(charging_set["kWhDelivered"].unique()))
print("siteID data range:\n", np.sort(charging_set["siteID"].unique()))
print("spaceID data range:\n", np.sort(charging_set["spaceID"].unique()))
print("stationID data range:\n", np.sort(charging_set["stationID"].unique()))
print("timezone data range:\n", np.sort(charging_set["timezone"].unique()))
print("userID data range:\n", np.sort(charging_set["userID"].unique()))
# print("connectionTime_date data range:\n", np.sort(charging_set["connectionTime_date"].unique()))
# print("connectionTime_time data range:\n", np.sort(charging_set["connectionTime_time"].unique()))
# print("disconnectTime_date data range:\n", np.sort(charging_set["disconnectTime_date"].unique()))
# print("disconnectTime_time data range:\n", np.sort(charging_set["disconnectTime_time"].unique()))
# print("doneChargingTime_date data range:\n", np.sort(charging_set["doneChargingTime_date"].unique()))
# print("doneChargingTime_time data range:\n", np.sort(charging_set["doneChargingTime_time"].unique()))

kWhDelivered data range:
 [  0.501        0.502        0.503      ...  77.7         89.36273194
 108.79724167]
siteID data range:
 [1 2]
spaceID data range:
 ['11900388' 'AG-1F01' 'AG-1F02' 'AG-1F03' 'AG-1F04' 'AG-1F05' 'AG-1F06'
 'AG-1F07' 'AG-1F08' 'AG-1F09' 'AG-1F10' 'AG-1F11' 'AG-1F12' 'AG-1F13'
 'AG-1F14' 'AG-3F15' 'AG-3F16' 'AG-3F17' 'AG-3F18' 'AG-3F19' 'AG-3F20'
 'AG-3F21' 'AG-3F22' 'AG-3F23' 'AG-3F24' 'AG-3F25' 'AG-3F26' 'AG-3F27'
 'AG-3F28' 'AG-3F29' 'AG-3F30' 'AG-3F31' 'AG-3F32' 'AG-3F33' 'AG-4F34'
 'AG-4F35' 'AG-4F36' 'AG-4F37' 'AG-4F38' 'AG-4F39' 'AG-4F40' 'AG-4F41'
 'AG-4F42' 'AG-4F43' 'AG-4F44' 'AG-4F45' 'AG-4F46' 'AG-4F47' 'AG-4F48'
 'AG-4F49' 'AG-4F50' 'AG-4F51' 'AG-4F52' 'CA-148' 'CA-149' 'CA-212'
 'CA-213' 'CA-303' 'CA-304' 'CA-305' 'CA-306' 'CA-307' 'CA-308' 'CA-309'
 'CA-310' 'CA-311' 'CA-312' 'CA-313' 'CA-314' 'CA-315' 'CA-316' 'CA-317'
 'CA-318' 'CA-319' 'CA-320' 'CA-321' 'CA-322' 'CA-323' 'CA-324' 'CA-325'
 'CA-326' 'CA-327' 'CA-489' 'CA-490' 'CA-491' 'CA-492' 'C

In [88]:
charging_set.drop('timezone', axis=1, inplace=True)
charging_set.head(3)

,id,kWhDelivered,siteID,spaceID,stationID,userID,WhPerMile,kWhRequested,milesRequested,minutesAvailable,...,doneChargingTime_year,doneChargingTime_month,doneChargingTime_day,modifiedAt_year,modifiedAt_month,modifiedAt_day,requestedDeparture_year,requestedDeparture_month,requestedDeparture_day,chargingDuration
0,5e23b149f9af8b5fe4b973cf,25.016,1,AG-3F30,1-1-179-810,194.0,250.0,25.0,100.0,463.0,...,2020.0,1.0,2.0,2020.0,1.0,2.0,2020.0,1.0,2.0,6.039167
1,5e23b149f9af8b5fe4b973d0,33.097,1,AG-1F01,1-1-193-825,4275.0,280.0,70.0,250.0,595.0,...,2020.0,1.0,2.0,2020.0,1.0,2.0,2020.0,1.0,2.0,9.025278
2,5e23b149f9af8b5fe4b973d1,6.521,1,AG-1F03,1-1-193-829,344.0,400.0,8.0,20.0,60.0,...,2020.0,1.0,2.0,2020.0,1.0,2.0,2020.0,1.0,2.0,10.713056


In [89]:
##### Warum haben wir uns das angeschaut? Können wir das rausnehmen?
# show example row of charging_set where user is not registered 
charging_set.loc[charging_set[charging_set['userID'] == 0].index[0]]

id                          5e23b149f9af8b5fe4b973da
kWhDelivered                                   0.812
siteID                                             1
spaceID                                      AG-1F14
stationID                                1-1-194-818
userID                                           0.0
WhPerMile                                        0.0
kWhRequested                                    -1.0
milesRequested                                  -1.0
minutesAvailable                                 0.0
modifiedAt                                       NaT
paymentRequired                                    0
requestedDeparture                               NaT
connectionTime_date              2020-01-02 00:00:00
connectionTime_time                         15:04:38
disconnectTime_date              2020-01-02 00:00:00
disconnectTime_time                         23:33:26
doneChargingTime_date            2020-01-02 00:00:00
doneChargingTime_time                       22

In [90]:
charging_set.loc[3765]

id                          5ea6278df9af8b4a2e7bd22c
kWhDelivered                                   4.725
siteID                                             1
spaceID                                      AG-1F10
stationID                                1-1-178-828
userID                                         651.0
WhPerMile                                      400.0
kWhRequested                                    20.0
milesRequested                                  50.0
minutesAvailable                               183.0
modifiedAt                       2020-04-10 14:55:16
paymentRequired                                 True
requestedDeparture               2020-04-10 17:58:07
connectionTime_date              2020-04-10 00:00:00
connectionTime_time                         14:55:07
disconnectTime_date              2020-04-10 00:00:00
disconnectTime_time                         15:43:03
doneChargingTime_date                            NaT
doneChargingTime_time                         

In [24]:
# charging_set.to_csv('data/cleaned_charging_set.csv', index=False)